In [ ]:
pip install opencc

     |████████████████████████████████| 765 kB 13.1 MB/s 


In [ ]:
pip install bpemb

     |████████████████████████████████| 1.2 MB 20.0 MB/s 


In [ ]:
from bpemb import BPEmb

In [ ]:
bpemb_en = BPEmb(lang="en", dim=100,vs=100000)
bpemb_zh = BPEmb(lang="zh", dim=100,vs=100000)

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs100000.model


100%|██████████| 1987533/1987533 [00:00<00:00, 28721361.08B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs100000.d100.w2v.bin.tar.gz


100%|██████████| 37969196/37969196 [00:00<00:00, 48894733.10B/s]


downloading https://nlp.h-its.org/bpemb/zh/zh.wiki.bpe.vs100000.model


100%|██████████| 1849493/1849493 [00:00<00:00, 25646205.10B/s]


downloading https://nlp.h-its.org/bpemb/zh/zh.wiki.bpe.vs100000.d100.w2v.bin.tar.gz


100%|██████████| 37914051/37914051 [00:00<00:00, 53349423.88B/s]


In [ ]:
bpemb_de = BPEmb(lang="de", dim=100,vs=100000)

downloading https://nlp.h-its.org/bpemb/de/de.wiki.bpe.vs100000.model


100%|██████████| 2068307/2068307 [00:00<00:00, 29602927.58B/s]


downloading https://nlp.h-its.org/bpemb/de/de.wiki.bpe.vs100000.d100.w2v.bin.tar.gz


100%|██████████| 38035802/38035802 [00:00<00:00, 50142937.35B/s]


In [ ]:
import tensorflow as tf
import jieba
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from opencc import OpenCC
import unicodedata
import re
import numpy as np
import os
import io
import time
import tensorflow_datasets as tfds

In [ ]:
class Encoder_cn(tf.keras.Model):
  def __init__(self, enc_units, batch_sz):
    super(Encoder_cn, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    # self.embedding = tf.keras.layers.Embedding(100000, 100)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    print('shape1111:  ')
    print('shape:  '+str(x.shape))
    print('type:  '+str(type(x)))
    print(x)
    
    # def printaa():
    #   tf.print('aaaaaaaa')
    # def my_numpy_func(a):
    #   tf.print('aa  :'+str(type(a)))
    #   a =  np.array(a)
    #   tf.print('aaa  :'+str(type(a)))
    #   # tf.print(s)
    #   emb=bpemb_zh.embed(bpemb_zh.decode_ids(a[0]))
    #   for i in range(1,64):
    #     b = bpemb_zh.embed(bpemb_zh.decode_ids(a[i]))
    #     emb= np.append(emb,b)
    #   emb = emb.reshape(64,15,100)
    #   return emb
    # @tf.function(input_signature=[tf.TensorSpec((64,15), tf.int64)])
    # def tf_function(input):
    #   y = tf.numpy_function(my_numpy_func, [input], tf.float32)
    #   return y 
    # printaa()
    # x= tf_function(x)

    # print('shape:  '+str(x.shape))
    # print('type:  '+str(type(x)))
    # print(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 隐藏层的形状 == （批大小，隐藏层大小）
    # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
    # 这样做是为了执行加法以计算分数  
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # 分数的形状 == （批大小，最大长度，1）
    # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
    # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
    attention_weights = tf.nn.softmax(score, axis=1)

    # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class Decoder_en(tf.keras.Model):
  def __init__(self, dec_units, batch_sz):
    super(Decoder_en, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    # self.embedding = tf.keras.layers.Embedding(100000, 100)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(100000)

    # 用于注意力
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # 编码器输出 （enc_output） 的形状 == （批大小，最大长度，隐藏层大小）
    context_vector, attention_weights = self.attention(hidden, enc_output)
    tf.print('context_vector  :'+str(context_vector))
    tf.print('context_vector  :'+str(context_vector.shape))
    # # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）

    # def my_numpy_func(a):
    #   a =  np.array(a)
    #   # tf.print(s)
    #   emb=bpemb_en.embed(bpemb_en.decode_ids(a[0]))
    #   for i in range(1,64):
    #     b = bpemb_en.embed(bpemb_en.decode_ids(a[i]))
    #     emb= np.append(emb,b)
    #   emb = emb.reshape(64,1,100)
    #   return emb
    # # @tf.function
    # def tf_function(input):
    #   y = tf.numpy_function(my_numpy_func, [input], tf.float32)
    #   return y 
    # x= tf_function(s)
    


    # x = self.embedding(x)
    # x=emb
    # x 在拼接 （concatenation） 后的形状 == （批大小，1，嵌入维度 + 隐藏层大小）
    # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）
    # x = self.embedding(x)
    # tf.print('embedding   '+str(x))
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    # 将合并后的向量传送到 GRU
    output, state = self.gru(x)

    # 输出的形状 == （批大小 * 1，隐藏层大小）
    output = tf.reshape(output, (-1, output.shape[2]))
    tf.print('output:shape '+str(output))

    # 输出的形状 == （批大小，vocab）
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
# example_input_batch, example_target_batch = next(iter(dataset_cn2en))
# example_input_batch.shape, example_target_batch.shape

In [ ]:
# sample_hidden = encoder_test.initialize_hidden_state()
# sample_output, sample_hidden = encoder_test(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
# print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [ ]:
# type(example_input_batch)

In [ ]:
# decoder_test = Decoder_en( units, BATCH_SIZE)

# sample_decoder_output, _, _ = decoder_test(emb_en_dec, sample_hidden, sample_output)

# print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [ ]:
d_model = 128
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=6000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.002, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
path_to_file = "drive/MyDrive/Colab Notebooks/cmn-clean.txt"
path_to_file_de = "drive/MyDrive/Colab Notebooks/deu.txt"
path_to_singleCorpus = "drive/MyDrive/Colab Notebooks/single_corpus_zh.txt"
path_to_newCorpus = "drive/MyDrive/Colab Notebooks/new_corpus.txt"
from google.colab import drive
drive.mount('drive')
input_file = open(path_to_file,"r",encoding="utf-8")

Mounted at drive


In [ ]:
cc = OpenCC('t2s')

In [ ]:
def preprocess_sentence(w):

    w = w.lower().strip()

    w = cc.convert(w)
 
    w = w.rstrip().strip()


    # w = re.sub(r"([?.!,:：？。，！])", r" \1 ", w)
    w = re.sub(r"[^\u4e00-\u9fa5\u0080-\uFFFF_a-zA-Z0-9?.!,:：？。，！']+", " ", w)

    w = w.rstrip().strip()
    w = '| ' + w + ' /'
    return w

In [ ]:
# preprocess_sentence(u"你大約遅了三天。")

In [ ]:
def create_dataset(path):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines]
    # print(word_pairs[200])

    return zip(*word_pairs)

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [ ]:
# def tokenize(lang):
#   lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
#   lang_tokenizer.fit_on_texts(lang)

#   tensor = lang_tokenizer.texts_to_sequences(lang)

#   tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
#   print (tensor),print(lang_tokenizer)
#   return tensor, lang_tokenizer

In [ ]:
def uniformLength_en_cn(list1,list2,maxlen,minlen):
  print(len(list1),len(list2))
  length = len(list1)
  a= 0
  l =0
  while length != l:
    l = len(list1)
    for i in range(a,length):
      if (len(bpemb_en.encode_ids(list1[i]))> maxlen or len(bpemb_en.encode_ids(list1[i]))< minlen
          or len(bpemb_zh.encode_ids(list2[i]))> maxlen or len(bpemb_zh.encode_ids(list2[i]))< minlen):
        del list1[i]
        del list2[i]
        length = len(list2)
        a=i
        break
      else:
        list1[i] = list1[i] + ' pad'*(maxlen-len(bpemb_en.encode_ids(list1[i])))
        list2[i] = list2[i] + ' pad'*(maxlen-len(bpemb_zh.encode_ids(list2[i])))
    
  # print(len(list1),len(list2))
  # print(list2[100])
  return list2,list1

In [ ]:
def cutDataset_en_cn(targ_lang,inp_lang,maxlen,minlen):
    targ_lang=list(targ_lang)
    inp_lang=list(inp_lang)
    targ_lang, inp_lang =uniformLength_en_cn(targ_lang, inp_lang,maxlen,minlen)
    return inp_lang, targ_lang

In [ ]:
def uniformLength_en_de(list1,list2,maxlen,minlen):
  print(len(list1),len(list2))
  length = len(list1)
  a= 0
  l =0
  while length != l:
    l = len(list1)
    for i in range(a,length):
      if (len(bpemb_en.encode_ids(list1[i]))> maxlen or len(bpemb_en.encode_ids(list1[i]))< minlen
          or len(bpemb_de.encode_ids(list2[i]))> maxlen or len(bpemb_de.encode_ids(list2[i]))< minlen):
        del list1[i]
        del list2[i]
        length = len(list2)
        a=i
        break
      else:
        list1[i] = list1[i] + ' pad'*(maxlen-len(bpemb_en.encode_ids(list1[i])))
        list2[i] = list2[i] + ' pad'*(maxlen-len(bpemb_de.encode_ids(list2[i])))

      
  # print(len(list1),len(list2))

  return list2,list1

In [ ]:
def cutDataset_en_de(targ_lang,inp_lang,maxlen,minlen):
    targ_lang=list(targ_lang)
    inp_lang=list(inp_lang)
    targ_lang, inp_lang =uniformLength_en_de(targ_lang, inp_lang,maxlen,minlen)
    return inp_lang, targ_lang

In [ ]:
en_cn, cn = create_dataset(path_to_file)
en_cn, cn =cutDataset_en_cn(en_cn, cn,15,1)
# en_de, de = create_dataset(path_to_file_de)
# en_de, de =cutDataset_en_de(en_de, de,15,5)

26828 26828


In [ ]:
# cn[3]

In [ ]:
# bpemb_zh.embed(cn[3133]).shape

In [ ]:
# cn_emb=bpemb_zh.embed(cn[0])
# for i in range(1,len(cn)):
# # for i in range(1,64):
#   a = bpemb_zh.embed(cn[i])
#   cn_emb= np.append(cn_emb,a)
# cn_emb = cn_emb.reshape(len(cn),15,100)


In [ ]:
# en_cn_emb=bpemb_en.embed(en_cn[0])
# for i in range(1,len(en_cn)):
# # for i in range(1,64):
#   a = bpemb_en.embed(en_cn[i])
#   en_cn_emb= np.append(en_cn_emb,a)
# en_cn_emb = en_cn_emb.reshape(len(en_cn),15,100)

In [ ]:
# cn_enc=bpemb_zh.encode_ids(cn[0])
# for i in range(1,len(cn)):
# # for i in range(1,10):
#   a = bpemb_zh.encode_ids(cn[i])
#   cn_enc= np.append(cn_enc,a)
# cn_enc = cn_enc.reshape(len(cn),15)


In [ ]:
# en_enc=bpemb_en.encode_ids(en_cn[0])
# for i in range(1,len(en_cn)):
# # for i in range(1,10):
#   a = bpemb_en.encode_ids(en_cn[i])
#   en_enc= np.append(en_enc,a)
# en_enc = en_enc.reshape(len(en_cn),15)

In [ ]:
# type(cn_enc)

In [ ]:
# def convert_cn(tensor):
#   for t in tensor:
#     if t!=-1:
#       print ("%d ----> %s" % (t, bpemb_zh.decode_ids([int(t)])))
# def convert_en(tensor):
#   for t in tensor:
#     if t!=6643:
#       print ("%d ----> %s" % (t, bpemb_en.decode_ids([int(t)])))
      

In [ ]:
# print ("Input Language; index to word mapping")
# convert_cn(bpemb_zh.encode_ids("| 你大约 ⁇ 了三天。 / pad pad pad pad pad pad pad"))
# print ()
# print ("Target Language; index to word mapping")
# convert_en(en_cn_tensor_train[222])

In [ ]:
# 采用 80 - 20 的比例切分训练集和验证集
en_cn_tensor_train, en_cn_tensor_val, cn_tensor_train, cn_tensor_val = train_test_split(en_cn, cn, test_size=0.1)

# 显示长度
print(len(en_cn_tensor_train), len(cn_tensor_train), len(en_cn_tensor_val), len(cn_tensor_val))

22995 22995 2555 2555


In [ ]:
BUFFER_SIZE = len(cn_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(cn_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
# vocab_inp_size = len(inp_lang.word_index)+1
# vocab_tar_size = len(targ_lang.word_index)+1

dataset_cn2en = tf.data.Dataset.from_tensor_slices((cn_tensor_train, en_cn_tensor_train)).shuffle(BUFFER_SIZE)
dataset_cn2en = dataset_cn2en.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
encoder_bpe_cn = Encoder_cn(units, BATCH_SIZE)
attention_layer_bpe_cn = BahdanauAttention(10)
decoder_bpe_en = Decoder_en(units, BATCH_SIZE)

In [ ]:
checkpoint_dir = './training_checkpoints_bpe_cn2en'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint_bpe_cn2en = tf.train.Checkpoint(optimizer=optimizer, encoder_bpe_cn=encoder_bpe_cn, decoder_bpe_en=decoder_bpe_en)

In [ ]:
# cn_emb[0].shape

In [ ]:
# x_de_inp = tf.expand_dims(bpemb_en.encode_ids("|") * BATCH_SIZE, 1)
# emb_en_dec=bpemb_en.embed(bpemb_en.decode_ids(x_de_inp[0].numpy()))
# for i in range(1,len(x_de_inp)):
#   a = bpemb_en.embed(bpemb_en.decode_ids(x_de_inp[i].numpy()))
#   emb_en_dec= np.append(emb_en_dec,a)
# emb_en_dec = emb_en_dec.reshape(64,1,100)

In [ ]:
# tf.expand_dims(bpemb_en.encode_ids("|") * BATCH_SIZE, 1)

In [ ]:
# def my_numpy_func_targ_0(s):
#   # tf.print('aa  :'+str(type(s)))
#   s =  np.array(s)
#   # tf.print(s)
#   emb=bpemb_en.embed(bpemb_en.decode_ids(s[0]))
#   for i in range(1,64):
#     a = bpemb_en.embed(bpemb_en.decode_ids(s[i]))
#     emb= np.append(emb,a)
#   emb = emb.reshape(64,1,100)
#   return emb
#  #(input_signature=[tf.TensorSpec(None, tf.float32)])
# @tf.function(input_signature=[tf.TensorSpec((64,1), tf.int32)])
# def tf_function_targ_0(input):
#   y = tf.numpy_function(my_numpy_func_targ_0, [input], tf.float32)
#   return y 


In [ ]:
# dec_inp = tf.expand_dims(bpemb_en.encode_ids("|") * BATCH_SIZE, 1)

In [ ]:
# dec_inp_emb = tf_function_targ_0(dec_inp)

In [ ]:
# for (batch, (inp, targ)) in enumerate(dataset_cn2en.take(1)):
#   x=inp
#   y=targ

In [ ]:
def my_numpy_func_targ(s):
  # tf.print(s.shape)
  # s =  np.array(s)
  # tf.print(len(s))
  emb=bpemb_en.embed(s[0].decode('utf-8'))
  for i in range(1,64):
    a = bpemb_en.embed(s[i].decode('utf-8'))
    emb= np.append(emb,a)
  emb = emb.reshape(64,15,100)
  return emb
@tf.function(input_signature=[tf.TensorSpec((64,), tf.string)])
def tf_function_targ(input):
  y = tf.numpy_function(my_numpy_func_targ, [input], tf.float32)
  return y 


In [ ]:
# @tf.function
def train_step_bpe_zh_en(inp,targ, enc_hidden):
  loss = 0
  with tf.GradientTape() as tape:
    print('start')
    targ = tf_function_targ(targ)
    tf.print('targ'+str(targ))
    tf.print(targ.shape)
    enc_output, enc_hidden = encoder_bpe_cn(inp, enc_hidden)
    print('end')
    print(enc_output)
    dec_hidden = enc_hidden
    print('teaching1')
    # dec_input = tf.expand_dims(bpemb_en.encode_ids("|") * BATCH_SIZE, 1)
    dec_input = targ[:,0]
    print('teaching2')
    # 教师强制 - 将目标词作为下一个输入
    for t in range(1, targ.shape[1]):
      # 将编码器输出 （enc_output） 传送至解码器
      print('teaching3')
      predictions, dec_hidden, _ = decoder_bpe_en(dec_input, dec_hidden, enc_output)
      print('teaching4')
      print('predictions:  '+str(predictions))
      print('dec_hidden:  '+str(dec_hidden))
      loss += loss_function(targ[:, t], predictions)
      print('loss'+str(loss))

      # 使用教师强制
      dec_input = tf.expand_dims(targ[:, t], 1)
      
      # dec_input = tf_function_targ(y_en)
      print('dec_input:  '+str(dec_input))
      print('teaching6')

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder_bpe_cn.trainable_variables + decoder_bpe_en.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
for (batch, (inp, targ)) in enumerate(dataset_cn2en.take(1)):
  x=inp
  y=targ

In [ ]:
aaa=tf_function_targ(targ)

In [ ]:
aaa.shape[0]

64

In [ ]:
def my_numpy_func(s):
  # tf.print('aa  :'+str(type(s)))
  # s =  np.array(s)
  # tf.print('ssssssss')
  # tf.print(s[0].decode('utf-8'))
  emb=bpemb_zh.embed(s[0].decode('utf-8'))
  for i in range(1,64):
    a = bpemb_zh.embed(s[i].decode('utf-8'))
    emb= np.append(emb,a)
  emb = emb.reshape(64,15,100)
  return emb
 #(input_signature=[tf.TensorSpec(None, tf.float32)])
@tf.function(input_signature=[tf.TensorSpec((64,), tf.string)])
def tf_function(input):
  y = tf.numpy_function(my_numpy_func, [input], tf.float32)
  return y 



In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
  
  start = time.time()

  enc_hidden = encoder_bpe_cn.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset_cn2en.take(steps_per_epoch)):
    # def my_numpy_func(a):
    #   tf.print('aa  :'+str(type(a)))
    #   a =  np.array(a)
    #   tf.print('aaa  :'+str(type(a)))
    #   # tf.print(s)
    #   emb=bpemb_zh.embed(bpemb_zh.decode_ids(a[0]))
    #   for i in range(1,64):
    #     b = bpemb_zh.embed(bpemb_zh.decode_ids(a[i]))
    #     emb= np.append(emb,b)
    #   emb = emb.reshape(64,15,100)
    #   return emb
    # @tf.function(input_signature=[tf.TensorSpec((64,15), tf.int64)])
    # def tf_function(input):
    #   y = tf.numpy_function(my_numpy_func, [input], tf.float32)
    #   return y 
    inp_x= tf_function(inp)
    # print(x.shape)

    batch_loss = train_step_bpe_zh_en(inp_x,targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
  # 每 2 个周期（epoch），保存（检查点）一次模型
  if (epoch + 1) % 2 == 0:
    checkpoint_bpe_cn2en.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))